In [113]:
import pandas as pd
import spacy
import re
import tldextract
from sklearn.preprocessing import OneHotEncoder
import numpy as np



nRowsRead = None  # specify 'None' if want to read whole file
postStrings = []
typeStrings = []

In [114]:
df1 = pd.read_csv('./input/mbti_1.csv', delimiter=',', nrows=nRowsRead)
df1.dataframeName = 'mbti_1.csv'

In [115]:
# making array of array of posts
for person,type in zip(df1['posts'],df1['type']):
    postStrings.append([type,person.split('|||')])

# removing excess single parentheses

for i in range(0,len(postStrings)):
    postStrings[i][1][0] = postStrings[i][1][0][1:]
    postStrings[i][1][-1] = postStrings[i][1][-1][:-1]

In [116]:
URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
DOMAIN_REGEX = "^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/?\n]+)"
COMMENT_REGEX = r"\[.*?\]"
ELLIPSIS_REGEX = r"^(\ *\.{3}\ *)|(\ *\.{3}\ *)$"

def urlReplace(stringToReplace):
	domain = tldextract.extract(stringToReplace.group())
	return domain.domain + '.' + domain.suffix


In [117]:
def containsOnlyUrlsOrNumbers(post):
	splits = post.split()
	for str in splits:
		if(not re.search(URL_REGEX,str) and not re.search('^\d+$',str) and not re.search('^\.\.\.+$',str)):
			return 1
	return 0

def containsComments(post):
	splits = post.split('] ')
	for str in splits:
		if(not re.search(COMMENT_REGEX,str)):
			return 1
		return 0

def removeEllipsis(post):
	return re.sub(ELLIPSIS_REGEX,'',post)

def removeWhitespace(post):
	post = " ".join(post.split())
	return post

def formatPerson(person):
	person[1] = list(map(lambda x : removeWhitespace(x),person[1]))
	person[1] = [post for post in person[1] if (
		containsOnlyUrlsOrNumbers(post) and containsComments(post))]
	person[1] = [removeEllipsis(post) for post in person[1]]
	person[1] = list(map(lambda x : re.sub(URL_REGEX, urlReplace, x),person[1]))
	return person

# for person in postStrings:
# 	for i in range(0, len(person)):
# 		person[i]=" ".join(person[i].split())
# 	person=[post for post in person if (containsOnlyUrlsOrNumbers(post) and containsComments(post))]
# 	person=[removeEllipsis(post) for post in person]
# 	for i in range(0, len(person)):
# 		person[i]=re.sub(URL_REGEX, urlReplace, person[i])

postStrings = list(map(formatPerson,postStrings))
# print(postStrings)
# for person in postStrings:
# 	print(person)
# 	print("================")
# 	for post in person:
# 		print(post)
# 		print("================================")


In [118]:
nlp = spacy.load('en_core_web_sm')


# docs = list(nlp.pipe(postStrings[0], n_process=-1))

docs = [[person[0],list(nlp.pipe(person[1], n_process=-1))] for person in postStrings]

# print([nlp.pipe(person,n_process=-1) for person in postStrings][0])

# print([token.text for token in docs[0][2]])

tokenizedPosts = []
personalityTypes = []
tokenDictionary = dict()
OHEcategories = []
labelIndex=[]

for person in docs:
  for post in person[1]:
    personalityTypes.append(person[0])
    tokenizedPosts.append([token.text for token in post])
    for token in post:
      tokenDictionary[token.text]=0

# tokenizedPosts = np.asarray(list(map(lambda x : np.asarray(x),tokenizedPosts)))
# personalityTypes = np.asarray(personalityTypes)
df = pd.DataFrame(columns=['type','posts'])

df['type'] = pd.Series(personalityTypes)
df['posts'] = pd.Series(tokenizedPosts)

OHEcategories = list(tokenDictionary.keys())

for i in range(len(OHEcategories)):
  labelIndex.append((i+1,OHEcategories[i]))

OneHt = pd.Series([';'.join(i) for i in df.posts]).str.get_dummies(';')

df = pd.concat([df.type,OneHt],axis=1)

df.head()
# df.to_csv('oneHotEncoded.csv', sep='\t')
